In [1]:
import tensorflow as tf
import os

base_dir = '../input/facerecognition30/dataset/'

In [2]:
IMAGE_SIZE = 224 #image size that we are going to set the images in the dataset to.
BATCH_SIZE = 64 #the number of images we are inputting into the neural network at once.

datagen = tf.keras.preprocessing.image.ImageDataGenerator( #preprocessing our image
    rescale = 1./255, #firstly, rescaling it to 1/255 which will make the file size smaller, hence reducing the training time
    validation_split=0.2 #secondly, normally a dataset has a test set and a training set, 
    #validation set is normally to test our neural network,which would give us a measure of accuracy on how well the neural network will do on the predictions.
    #here we are telling keras to use 20% for validation and 80% training
)

train_generator = datagen.flow_from_directory( #training generator
    base_dir, #the directory having the fruits and vegetable photos
    target_size=(IMAGE_SIZE, IMAGE_SIZE),  #converting images to 224 by 224
    batch_size = BATCH_SIZE, #images getting inputed into the neural network through each epoch or each step
    subset='training' #the name we will call it
)
val_generator = datagen.flow_from_directory(  #validation generator
    base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 12724 images belonging to 30 classes.
Found 3165 images belonging to 30 classes.


In [3]:
print(train_generator.class_indices)
labels = '\n'.join(sorted(
    train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
    f.write(labels)

{'Courteney_Cox': 0, 'David_Schwimmer': 1, 'Jennifer_Aniston': 2, 'Jim_Parsons': 3, 'Johnny_Galeck': 4, 'Kunal_Nayya': 5, 'Lisa_Kudrow': 6, 'Matt_LeBlanc': 7, 'Matthew_Perry': 8, 'Pankaj_Tripathi': 9, 'ROHIT_SHARMA': 10, 'Sachin_Tendulka': 11, 'Simon_Helberg': 12, 'aishwarya_rai': 13, 'angelina_jolie': 14, 'arnold_schwarzenegger': 15, 'bhuvan_bam': 16, 'brad_pitt': 17, 'dhoni': 18, 'hardik_pandya': 19, 'manoj_bajpayee': 20, 'messi': 21, 'mohamed_ali': 22, 'pewdiepie': 23, 'random_person': 24, 'ronaldo': 25, 'scarlett_johansson': 26, 'suresh_raina': 27, 'sylvester_stallone': 28, 'virat_kohli': 29}


In [4]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE, include_top = False, weights='imagenet')

2022-01-04 21:05:18.972763: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 21:05:19.069218: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 21:05:19.069945: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 21:05:19.071565: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

9420800/9406464 [==============================] - 0s 0us/step


In [5]:
base_model.trainable = False #this freezes all the neurons for our base model
model = tf.keras.Sequential([ #neural networks act in a sequence of layers, so we add layers as we want
  base_model,
  tf.keras.layers.Conv2D(32,3, activation = 'relu'), #This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs. Bascially, it trying to understand the patterns of the image
  tf.keras.layers.Dropout(0.2), #This layer prevents Neural Networks from Overfitting, i.e being too precise to a point where the NN is only able to recognize images that are present in the dataset
  tf.keras.layers.GlobalAveragePooling2D(), #This layer calculates the average output of each feature map in the previous layer, thus reducing the data significantly and preparing the model for the final layer
  tf.keras.layers.Dense(30,activation='softmax')
])

In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(), #Adam is a popular optimiser, designed specifically for training deep neural networks
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [7]:
epochs = 10 #higher the epochs, more accurate is the NN, however it could cause Overfitting, if too high
history = model.fit(
    train_generator, 
    epochs = epochs, 
    validation_data=val_generator
)

2022-01-04 21:05:23.937431: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-01-04 21:05:27.658105: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


199/199 [==============================] - 141s 666ms/step - loss: 2.7450 - accuracy: 0.2683 - val_loss: 2.1512 - val_accuracy: 0.3975
Epoch 2/10
199/199 [==============================] - 67s 334ms/step - loss: 1.9064 - accuracy: 0.4561 - val_loss: 1.6809 - val_accuracy: 0.5283
Epoch 3/10
199/199 [==============================] - 65s 329ms/step - loss: 1.5143 - accuracy: 0.5605 - val_loss: 1.4376 - val_accuracy: 0.5886
Epoch 4/10
199/199 [==============================] - 65s 324ms/step - loss: 1.2928 - accuracy: 0.6255 - val_loss: 1.2889 - val_accuracy: 0.6367
Epoch 5/10
199/199 [==============================] - 66s 334ms/step - loss: 1.1073 - accuracy: 0.6721 - val_loss: 1.2018 - val_accuracy: 0.6632
Epoch 6/10
199/199 [==============================] - 66s 330ms/step - loss: 0.9753 - accuracy: 0.7098 - val_loss: 1.1359 - val_accuracy: 0.6859
Epoch 7/10
199/199 [==============================] - 66s 334ms/step - loss: 0.9051 - accuracy: 0.7313 - val_loss: 1.1332 - val_accuracy: 0.

In [8]:
#now that we have our neural network trained with tensorflow and keras, we can export it 
saved_model_dir = '' #means current directory
tf.saved_model.save(model, saved_model_dir) #saves to the current directory

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) 
tflite_model = converter.convert() #converts our model into a .tflite model which flutter uses for ondevice machine learning

with open('model.tflite', 'wb') as f: #to write the converted model into a file, written as binary so add 'wb' instead of 'w'
  f.write(tflite_model)

2022-01-04 21:18:21.303919: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-01-04 21:18:42.979447: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 21:18:42.979813: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-01-04 21:18:42.979999: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-01-04 21:18:42.980461: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 21:18:42.980854: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had ne